In [171]:
import os
import glob
from pathlib import Path
import re
import pandas as pd
from datetime import datetime
file_list = Path('热搜数据').glob('**/*.md')
morning_list = list(filter(lambda x: str(x).find('11')!=-1 & str(x).find('checkpoint')==-1, file_list))
evening_list = list(filter(lambda x: str(x).find('23点')!=-1 & str(x).find('checkpoint')==-1, file_list))

In [177]:
def collect_md_to_pd(file_list):
    df = pd.DataFrame(columns=['时间','热搜','热度'])
    for file in file_list:
        with open(file) as f:
            lines = f.readlines()
            lines = [i.strip() for i in lines]
            data = list(filter(None, lines))
            del data[0]
            data = data[0:100]
            rank = data[1::2]
            content = data[::2]
            time = re.findall(r'日/(.+)\.', str(file))[0]
            for i in range(len(content)):
                content[i] = re.findall('、(.+)', content[i])[0]
            for i in range(len(rank)):
                rank[i] = re.findall(' (.+)', rank[i])[0]
            dict = {'热搜': content, '热度': rank}
            df1 = pd.DataFrame(dict)
            df1.insert(0, '时间', time)
            df = df.append(df1)
    return df

In [180]:
from openpyxl import load_workbook
path = '总结.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
new = collect_md_to_pd(morning_list).sort_values(by='时间', ascending=True).reset_index(drop=True)
date = datetime.now().strftime('%y-%m-%d %H-%M-%S')
new.to_excel(writer, sheet_name=date, index=False)
writer.save()

In [ ]:
df.to_excel('总结.xlsx', sheet_name=date, index=False)